In [ ]:
import cv2
import numpy as np

src1 = cv2.imread("image1_1.jpg")
src1 = cv2.


In [3]:
kernel = np.ones((11, 11), np.uint8) 
dst = cv2.morphologyEx(src1, cv2.MORPH_BLACKHAT, kernel) 
cv2.imshow("src1", src1)
cv2.imshow("after blackhat", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()